
Interactive theorem proving is cool.

Automated theorem proving is really cool.

It does happen though that you can get outside the ability of an automated theorem prover in one shot. It may need breadcrumbs or guidance along the way.
But then you need some kind of scaffolding to organize these hints.

I think this is an interesting design I don't see much.

The basic premise is that we allow a single inference rule SUPERMODUS (TM) which uses an automated prover to discharge a big step of the proof. We get to manually select if we like the hypothesies it has access to.



unsat core might be useful to prune the hypotheses for faster future proofs


In [2]:
from typing import Any, Tuple, List
from z3 import *
Form = Any
Thm = Tuple[int, Form]
BoolRef.__and__ = lambda self, other: And(self, other)
BoolRef.__or__ = lambda self, other: Or(self, other)
BoolRef.__invert__ = lambda self: Not(self)
BoolRef.__gt__ = lambda self, other: Implies(self, other) # maybe not worth it. It twists meaning too much
def QForAll(vars, hyp, conc): # useful helper pattern
    return ForAll(vars, Implies(hyp, conc))

def check(thm : Thm):
    hsh, form = thm
    assert hsh == hash(("secret",form))

# recording all known theorems makes certain big hammer autos possible.
# thms = [] # or set()
def formula(thm : Thm) -> Form:
    return thm[1]

def trust(form : Form) -> Thm:
    #thms.append(concthm)
    return hash(("secret",form)), form

def infer(hyps : List[Thm], conc : Form, timeout=1000) -> Thm:
    s = Solver()
    for hyp in hyps:
        check(hyp)
        s.add(hyp[1])
    s.add(Not(conc))
    s.set("timeout", timeout)
    res = s.check()
    if res != z3.unsat:
        print(s.sexpr())
        if res == z3.sat:
            print(s.model())
        assert False,res

    return trust(conc)

def extend(hyps, x,y, P : Form, prefix="c"):
    infer(hyps, ForAll([x],Exists([y], P)))
    f = FreshConst(ArraySort(x.sort(), y.sort()), prefix=prefix)
    return f, trust(ForAll([x], P.substitute(y,f[y])))

# A primitive definition mechanism. Maybe + Lambda this is good enough?
def define(name, x):
    f = FreshConst(x.sort(), prefix=name)
    return f, trust(f == x)




In [ ]:
proof_db = []
Thm = int

def trust(form : Form, reasons = None) -> Thm:
    proof_db.append((form, reasons))
    return len(proof_db)-1

def check(form):
    pass

def formula(thm:Thm) -> Form: # this replaces "check" I guess if check returns the formula and does the check.
    return proof_db[thm][0]

def infer(hyps : List[Thm], conc : Form, timeout=1000) -> Thm:
    s = Solver()
    for hyp in hyps:
        s.add(formula(hyp))
    s.add(Not(conc))
    s.set("timeout", timeout)
    res = s.check()
    if res != z3.unsat:
        print(s.sexpr())
        if res == z3.sat:
            print(s.model())
        assert False,res

    return trust(conc, reasons=("infer", hyps))


def extend(hyps, x,y, P : Form, prefix="c"):
    infer(hyps, ForAll([x],Exists([y], P)))
    f = FreshConst(ArraySort(x.sort(), y.sort()), prefix=prefix)
    return f, trust(ForAll([x], P.substitute(y,f[y])), reasons=("extend", hyps))

# A primitive definition mechanism. Maybe + Lambda this is good enough?
def define(name, x):
    f = FreshConst(x.sort(), prefix=name)
    return f, trust(f == x, reasons = ("define",))

def metamath():
    # add infer as a metamath rule? and use metamath syntax otherwise.
    # there's a lot of impedance mismatch.
    pass
def json_proof():
    json.dump(proof_db, open("proof_db.json", "w"))
def sql():
    for (thm, reasons) in proof_db:
        conn.exec("INSERT INTO thms (thm, reasons) VALUES (?,?)", thm, reasons)

None
[]
False


cnf(c0,plain,foo(bar,Biz)).

Evidence.

Z3 handles a quantifier free fragment best. (maybe a single forall isn't)
PRA is quantifier free
So was ACL2
Check out ACL2

Carrying "proof" objects as last argument a la datalog.
antiderivative is proof of integration
Dual vector (although maybe not necessary)
List of zeros or poles

Don't ever pack existentials.
evenness and the div2 number
Does this get out of control?


In [14]:
import subprocess
def infer_vampire(hyps : List[Thm], conc : Form, timeout=1000) -> Thm:
    s = Solver()
    for hyp in hyps:
        check(hyp)
        s.add(hyp[1])
    s.add(Not(conc))
    with open("/tmp/temp.smt2", "w") as f:
        f.write(s.sexpr())
    print(s.sexpr())
    res = subprocess.run(["vampire", "/tmp/temp.smt2", "--output_mode", "smtcomp"], timeout=1.0, capture_output=True)
    print(res)
    if "unsat" not in res.stdout.decode("utf-8"):
        print(res.stdout.decode("utf-8"))
        assert False
    return trust(conc)

infer_vampire([], BoolVal(True))
infer_vampire([], BoolVal(False))


    


(assert (not true))

CompletedProcess(args=['vampire', '/tmp/temp.smt2', '--output_mode', 'smtcomp'], returncode=0, stdout=b'Running in auto input_syntax mode. Trying SMTLIB2\nunsat\n', stderr=b'')
(assert (not false))

CompletedProcess(args=['vampire', '/tmp/temp.smt2', '--output_mode', 'smtcomp'], returncode=0, stdout=b'Running in auto input_syntax mode. Trying SMTLIB2\nsat\n', stderr=b'')
Running in auto input_syntax mode. Trying SMTLIB2
sat



AssertionError: 

In [25]:
import egglog
egraph = egglog.EGraph()
egglog.bindings.Function("foo").register(egraph)

TypeError: argument 'decl': 'str' object cannot be converted to 'FunctionDecl'

In [41]:
from egglog.bindings import *


#egraph.register(Function(FunctionDecl("foo", Schema([], "i64"))))
#egraph.relation("foo", "i64")
help(egraph.function)


Help on method function in module egglog.egraph:

function(*args, **kwargs) -> 'Any' method of egglog.egraph.EGraph instance
    Registers a function.



# Tactics

There are a number of derived rules. We can reuse z3's tactics. This is a bit of slight of hand.

In [47]:
def exI(x, P : Thm) -> Thm:
    return infer([P], Exists([x], P[1]))

A = DeclareSort("A")
a = Const("a", A)
P = Function("P", A, BoolSort())
ax = trust(P(a))
exI(a, ax)


# maybe
def simp(thm : Thm):
    check(thm)
    return infer([thm],z3.simplify(thm[1]))

x = Int("x")
thm = infer([], x + 0 == x)
simp(thm)

# other tactics
def tactic(name, thm):
    g = Goal(thm[1])
    conc = z3.Tactic(name)(g)
    print(conc)
    return infer([thm], conc)

tactic("simplify", thm)
    


[[]]


Z3Exception: True, False or Z3 Boolean expression expected. Received [[]] of type <class 'z3.z3.ApplyResult'>

In [46]:
z3.tactics()

['ackermannize_bv',
 'subpaving',
 'horn',
 'horn-simplify',
 'nlsat',
 'qfnra-nlsat',
 'qe-light',
 'nlqsat',
 'qe',
 'qsat',
 'qe2',
 'qe_rec',
 'psat',
 'sat',
 'sat-preprocess',
 'ctx-solver-simplify',
 'psmt',
 'unit-subsume-simplify',
 'aig',
 'add-bounds',
 'card2bv',
 'degree-shift',
 'diff-neq',
 'eq2bv',
 'factor',
 'fix-dl-var',
 'fm',
 'lia2card',
 'lia2pb',
 'nla2bv',
 'normalize-bounds',
 'pb2bv',
 'propagate-ineqs',
 'purify-arith',
 'recover-01',
 'bit-blast',
 'bv1-blast',
 'bv_bound_chk',
 'propagate-bv-bounds',
 'propagate-bv-bounds2',
 'reduce-bv-size',
 'bv-slice',
 'bvarray2uf',
 'dt2bv',
 'elim-small-bv',
 'max-bv-sharing',
 'blast-term-ite',
 'cofactor-term-ite',
 'collect-statistics',
 'ctx-simplify',
 'demodulator',
 'der',
 'distribute-forall',
 'dom-simplify',
 'elim-term-ite',
 'elim-uncnstr2',
 'elim-uncnstr',
 'elim-predicates',
 'euf-completion',
 'injectivity',
 'snf',
 'nnf',
 'occf',
 'pb-preprocess',
 'propagate-values2',
 'propagate-values',
 'reduc

In [43]:
x = Bool("x")
print( x & x > ~x | x)
x = Array("x", StringSort(), StringSort())
print(x.sort())
print(x.sort().domain())
print(x.sort().range())
print(dir(x.sort()))
x.sort().name()

Implies(And(x, x), Or(Not(x), x))
Array(String, String)
String
String
['__bool__', '__class__', '__copy__', '__deepcopy__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_repr_html_', 'as_ast', 'ast', 'cast', 'ctx', 'ctx_ref', 'domain', 'domain_n', 'eq', 'get_id', 'hash', 'kind', 'name', 'range', 'sexpr', 'subsort', 'translate', 'use_pp']


'Array'

In [ ]:
class Backward():
    def __init__(self, goal):
        self.goal = infer([], goal > goal)
    def intros(self):
        pass
class Forward(): #Isar
    def __init__(self, goal):
        self.goal = goal
        self.lemmas = []
        self.vars = []
        self.hyps = []
    def intro(self,vars):
        self.vars.extend(vars)
        return self
    def assume(self, hyps):
        self.hyps.extend(hyps)
        return self
    def wrap(self, form):
        return ForAll(self.vars, Implies(And(self.hyps), form))
    def have(self, conc, by=[]):
        self.lemmas.append(infer(by + self.lemmas, conc))
        return self
    def qed(self):
        return infer(self.lemmas, self.goals)





A Section or locale or module mechanims is pretty similar to an Isar proof. We tend t make things stored whereas proofs have anonymus



In [ ]:
class Section(): # locale?
    def _init__(self, name, type_params=[A]):
        self.fixes = []
        self.assumes = []
        self.defs = []
    def fix(self, x):
        self.fixes.append(x)
        return self
    def assume(self, x):
        self.assumes.append(x)
        return self
    def define(self, x):
        self.defs.append(x)
        return self
    def instance(self, fixes,assumes):
    def sublocale(self):
        # extend a locale


lattice = Section("lattice")\
    .fixes("x", )\
    .fixes("meet", ArraySort())\
    .fixes("join", )\
    


https://isabelle.in.tum.de/library/HOL/HOL/Orderings.html
Is a decent root
locale partial_preordering =
  fixes less_eq :: ‹'a ⇒ 'a ⇒ bool› (infix ‹❙≤› 50)
  assumes refl: ‹a ❙≤ a› ― ‹not ‹iff›: makes problems due to multiple (dual) interpretations›
    and trans: ‹a ❙≤ b ⟹ b ❙≤ c ⟹ a ❙≤ c›

locale preordering = partial_preordering +
  fixes less :: ‹'a ⇒ 'a ⇒ bool› (infix ‹❙<› 50)
  assumes strict_iff_not: ‹a ❙< b ⟷ a ❙≤ b ∧ ¬ b ❙≤ a›
begin

Hmm. Obvously isabelle uses it's parametricity.
I guess the only thing to do is use meta parametricity.
Could instantiate with sort to sanity check def and then run all proofs at instnatiation. Or could elide checking process if we _really_ trust this meta reasoning


There's just so much boiler plate. Ugh

Another option is to pick a universal sort. And have injections in and out 





In [ ]:
# locales as functions. But how do we sanity check at definition time?
def partial_ordering(A,leq,axioms):
    x = FreshConst(A, prefix="x")
    refl = infer( axioms, ForAll([x], leq(x,x)))

def partial_ordering(sorts, sig, axioms):
    A = sorts["A"]
    leq = sig["leq"]
    x = FreshConst(sorts["A"], prefix="x")
    refl = infer()

def partial_ordering(ctx):
    x = FreshConst(ctx.A, prefix="x")
    refl = infer( axioms, ForAll([x], leq(x,x)))

class Partial_Ordering():
    sig = # fixes 
    axioms = # assumes
    def instan(self):
        pass


In [ ]:
# Can we embed shallower into python class system?
# it becomes a pattern rather than a library?
# Ughh. I just know this is getting hairy and silly for no reason
class Locale():
    def __init__(cls,name,bases,clsdict) -> None:
        super().__init__(name,bases,clsdict)
        cls.fixes = []
        cls.assumes = []
        cls.defs = []

# use decorator
@locale
class Partial_Preordering(metaclass=Locale): # metaclass is so funky.
    leq = Function("leq",A,A,BoolSort())
    fixes = [Function("leq", A, A, BoolSort())]
    assumes = {"refl":   }
    def __init__(self, A, leq):
        leq = Function("leq",A,A,BoolSort())
        fixes = [Function("leq", A, A, BoolSort())]
        assumes = {"refl":   }

# is initializing the class the same and intepreting the locale?
# How 
class Preordering(Partial_Preordering):
    def __init__(self, A, leq, ):
        assert leq.sort() == ArraySort(A,A,BoolSort())



In [119]:
z3.simplify(IntVal(3) / 2)

1

# Reals
The z3 "reals" are more like algebraic numbers.
We can add some kind of induction axiom.
Or define CReal as a sequence $ \bN -> bR $ that converges.

There isn't much to talk about in analysis without discussing functions $\bR -> \bR$ or $\bN -> \bR$

## Cosine Doesn't Exist
The notion of completed infinities is problematic and shows up everywhere. Problems that are ultimately obviously solvable, albeit by massive enumeration are nice.
"defining" cosine to be the 1000th term expansion mod 2 pi is pretty good.
Metatheoretically, we can establish the connection to "actual" cosine if we like.
Comparing float32 to float64 is a reasonable method for establishing accuracy estimates / convergence estimates.
The weakness is that sometimes the 1000th term expansion is not an acceptable approxiamtion and we can only detect this metathoretically.
The statement "forall integers" does seem like it is touching to close to completing the integers. Cody mentioned that there is a "cut elimination" property for induction that once you want to apply said theorem to an actual concrete integer, you can unwind the induction a finite number of steps.
Z3 could solve this is 1 shot in principle. It is interesting to maybe stick to proof methods that are just guiding what should be doable in 1 shot.

Z3 won't like variable pow.


Simplest challenge problem
lim(n->inf, 1/n) = 0



In [66]:
exp = Array("exp", IntSort(), ArraySort(RealSort(), RealSort())) # Array("cos", RealSort(), IntSort(), RealSort()) # returns a CReal
exp0def = ForAll([x],exp[0][x] == 1)
fact = lambda x: Real(x)
expsuccdef = QForAll([n,x], n > 0, exp[n][x] == x**n / fact(n) + exp[n-1][x]) # cosine is different on even odd. Yuck.

# expansion example
infer([exp_def], ForAll([x], exp[2][x] == 1 + x + x**2)) 

# comparative accuracy. Don't even fully expand
diff = infer([exp_def], ForAll([x], exp[2][x] - exp[1][x] == x**2 ))
infer([diff], QForAll([x], 0 <= x & x <= 1,  exp[2][x] - exp[1][x] <= 0.1))


Z3Exception: b'parser error'

In [20]:
RealSeq = ArraySort(IntSort(), RealSort())
a = Const("a", RealSeq)
x,eps = Reals("x eps")
n,m = Ints("n m")
#convergent = define("convergant", Lambda([x], Exists([y], ForAll([n], x[n] == y))))

norm_bound, norm_bound_def = define("norm_bound", Lambda([x,eps], (x <= eps) & (-eps <= x)))
abs, abs_def = define("abs", Lambda([x], If(x >= 0, x, -x)))
lim, lim_def = define("lim", Lambda([a,x], 
               ForAll([eps], Exists([n], QForAll([m], m > n, norm_bound(x - a[m], eps))))))
# skolemize it probably.
n = Array("n", RealSort(), IntSort())
# convergence
lim, lim_def = define("lim", Lambda([a,x,n], 
               ForAll([eps], QForAll([m], m > n[eps], norm_bound[x - a[m], eps]))))

lim[Lambda([m], If(m >=0, 1/m, 0)), RealVal(0), Lambda([eps], 1/eps)]

TypeError: 'ArrayRef' object is not callable

In [ ]:
# This is kind of bishop's definition of just picking 1/n for the modulus of convergence
harmonic_lin = define("linharm", Lambda([a,x], ForAll([eps], QForAll([m], m > 0, norm_bound(x - a[m], 1/m)))))
cauchy = define("cauchy", Lambda([a,n], ForAll([eps], QForAll([m], m > n[eps], norm_bound(a[m] - a[n], eps)))))

define("cauchy", Lambda([a,n], QForAll([eps, m], m > n[eps], norm_bound(a[m] - a[n], eps))))


R = DeclareSort("R")
eclass = Function("eclass", IntSeq, R) # Quotient by cauchy equivalence
QForAll([a,b], ForAll([n], norm_bound(a[n] - b[n]) <= 1/n),    
               eclass(a) == eclass(b) )




In [75]:
#SortRef.__rshift__ = lambda self, other: ArraySort(self, other) # Nah bad idea. wrong associativty
def ArrowSort(*args):
    if len(args) == 1:
        return args[0]
    return ArraySort(args[0], ArrowSort(*args[1:]))
SortRef.__mul__ = lambda self, other: TupleSort(f"Pair_{self}_{other}", [self, other])[0]
IntSort() >> IntSort() >> IntSort()
IntSort() * IntSort()
ArrowSort(IntSort(), IntSort(), IntSort())

Array(Int, Array(Int, Int))

In [ ]:
# Open-closed induction. We can extend any point some epsilon out, and we can close any open ball back to closed.
def induct_real(P : Form) -> Thm:
    assert P.sort().name() == "Array"
    assert P.sort().domain() == RealSort()
    assert P.sort().range() == BoolSort()
    n = FreshConst(RealSort())
    eps = FreshConst(RealSort())
    hyp = P[0] & \
         ForAll([n], Exists([eps], P[n] & (eps > 0) > P[n + eps])) & \
         ForAll([n], ForAll([eps], P[x - eps] & (eps > 0) > P[x]))
    #------------------------------------------
    conc =  ForAll([n], P[n])
    return trust(hyp > conc)
# generalization to general topology?

In [ ]:
# https://math.stackexchange.com/questions/4202/induction-on-real-numbers
# https://arxiv.org/abs/1208.0973
induct_real = trust(QForAll([P], 
                QForAll([x],P[x] , Exists([y], y > x & P[y])) &
                QForAll([x,y], QForAll([y], y < x, P[y]), P[x])),  
                QForAll([x],x >= 0, P[x]))

In [40]:
";(declare-datatypes ((Nat 0)) (((zero) (succ (pred Nat)))))"
smtlib = """
;(declare-datatypes () ((Nat (zero)))) ; (succ (pred Nat)))))
(declare-datatype Nat ((zero) (succ (pred Nat))))
;(declare-datatypes ((Nat 0)) (((zero) (succ (pred Nat)))))
(declare-fun add (Nat Nat) Nat)
(declare-fun x () Nat)
(assert (forall ((x Nat)) (= (add zero x) x)))
(assert (forall ((x Nat) (y Nat)) (= (add (succ x) y) (succ (add x y)))))
(assert (let ((a!1 (and (forall ((y Nat)) (= (add zero y) (add y zero)))
                (forall ((c!13 Nat))
                  (let ((a!1 (not (forall ((y Nat))
                                    (= (add c!13 y) (add y c!13)))))
                        (a!2 (forall ((y Nat))
                               (= (add (succ x) y) (add y (succ x))))))
                    (or a!1 a!2))))))
  (or (not a!1) (forall ((c!13 Nat) (y Nat)) (= (add c!13 y) (add y c!13))))))
(assert (not (forall ((x Nat) (y Nat)) (= (add x y) (add y x)))))
(check-sat)
"""

open("/tmp/peano.smt2", "w").write(smtlib)


894

In [41]:
!vampire /tmp/peano.smt2

perf_event_open failed (instruction limiting will be disabled): Permission denied
% Running in auto input_syntax mode. Trying SMTLIB2


^C
28863 Aborted by signal SIGINT on /tmp/peano.smt2
% ------------------------------
% Version: Vampire 4.6.1 (commit af1735c99 on 2021-12-01 14:43:47 +0100)
% Linked with Z3 4.8.13.0 f03d756e086f81f2596157241e0decfb1c982299 z3-4.8.4-5390-gf03d756e0
% Termination reason: Unknown
% Termination phase: Saturation

% Memory used [KB]: 29551
% Time elapsed: 24.705 s
% ------------------------------
% ------------------------------


# HOL

So HOL is supposed to be weaker than set theory but still very expressive. 
See 
- Andrews book
- HOL Light tutorial
- Gordon paper


In [ ]:
o = BoolSort()
i = DeclareSort("i")
def EQ(A,B):


TRUE = None
FALSE = None

# Set Theory



In [44]:

# Zf
Set = DeclareSort("Set")
elem = Function("elem", Set, Set, BoolSort())
A,B,C,x,y,z = Consts("A B C x y z", Set) 

ax_emp = trust(Exists([A],ForAll([x], ~elem(B,A))))
ax_pair = trust(ForAll([A,B], Exists([C], ForAll([x], elem(x, C) == elem(x,A) | elem(x,B)))))
ax_ext = trust(ForAll([A,B], ForAll([x], elem(x,A) == elem(x,B)) == (A == B)))
ax_union = trust(ForAll([A], Exists([B], ForAll([x], elem(x,B) == Exists([y], elem(x,y) & elem(y,A))))))
def ax_sep(B,P):
    A = FreshConst(Set)
    return trust(Exists([A], ForAll([x], elem(x,A) == elem(x,B) & P(x))))

# ordered pairs

# injectivity theorem


In [ ]:
def extend(hyps,x,P, prefix):
    infer(hyps, Exists([x], P))
    s = FreshConst(Set, prefix=prefix)
    return s, trust(P.substitute(s, x))

# Ordinals



In [ ]:
# Constructive Real

# Z3 reals are more like algebraic numbers, not reals.
CReal = ArraySort(Nat, RealSort())
eq = Function("eq", CReal, CReal, BoolSort())
ax_eq = trust(ForAll([x,y], eq(x,y) == (ForAll([n], x[n] - y[n] < 1/n))

# Category
Topos Book
Scott Encoding using exists predicate.
Well formed as precondition.

In [ ]:
Arr = DeclareSort("Arr")
Ob = DeclareSort("Ob")
ex = Function("Ex", Arr, BoolSort())

f,g,h = Consts("f g h", Arr)
A,B,C = Consts("A B C", Ob)
comp = Function("comp", Arr, Arr, Arr)
id_ = Function("id", Ob, Arr)
dom = Function("dom", Arr, Ob)
cod = Function("cod", Arr, Ob)
z3.ExprRef.__lshift__ = lambda self, other: comp(self, other)
#ax_comp = trust(ForAll([f,g,h], comp(f,comp(g,h)) == comp(comp(f,g),h)))
ax_comp_assoc = trust(ForAll([f,g,h], f << (g << h) == (f << g) << h))
#ax_id_l = trust(ForAll([f,A], (cod(f) == A) > (id_(A) << f == f)))
#ax_id_r = trust(ForAll([f,A], (dom(f) == A) > (f << id_(A) == f)))
ax_id_l = trust(ForAll([f], id_(cod(f)) << f == f)) # inline the dom/cod
ax_id_r = trust(ForAll([f], f << id_(dom(f)) == f)) 
ax_cat = [ax_id_l, ax_id_r, ax_comp_assoc]

ax_dom_id = trust(ForAll([A], dom(id_(A)) == A))
ax_cod_id = trust(ForAll([A], cod(id_(A)) == A))
ax_ex_id = trust(ForAll([A], ex(id_(A))))
ax_ex_comp = trust(ForAll([f,g], ex(f << g) == (dom(f) == cod(g))))
ax_wff = [ax_dom_id, ax_cod_id, ax_ex_id, ax_ex_comp]

junk = Const("junk", Arr)
ax_junk = trust(ForAll([f], Implies(ex(f), f != junk))) # bidirectional?

wff_id_id = infer(ax_wff + ax_cat, ex(id_(A) << id_(A)) & (dom(id_(A) << id_(A)) == A) & (cod(id_(A) << id_(A)) == A))
wff_id_id
#infer(ax_cat + ax_wff + [wff_id_id], id_(A) << id_(A) == id_(A))

# sanity check
#infer(ax_cat + ax_wff, False)

monic, monic_def = define("monic", Lambda([f], ForAll([g,h], Implies(f << g == f << h, h == g))))
epic, epic_def = define("epic", Lambda([f], ForAll([g,h], Implies(g << f == h << f, h == g))))

In [59]:

id_monic = infer(ax_cat + ax_wff + [monic_def], ForAll([A], monic[id_(A)]))
id_monic

(-6531872273261288051, ForAll(A, monic!200[id(A)]))

In [ ]:
# pullback is universal square 
pullback = define("pullback", Lambda([f,g,h,k, univ], 
f << g == h << k & 
ForAll([x,y], Exists([univ], Implies(x << g == y << k, x == univ << g  & y == univ << k)))))

In [56]:
infer(ax_cat + ax_wff + [monic_def], False)

(declare-sort Arr 0)
(declare-sort Ob 0)
(declare-fun monic!89 () (Array Arr Bool))
(declare-fun comp (Arr Arr) Arr)
(declare-fun id (Ob) Arr)
(declare-fun cod (Arr) Ob)
(declare-fun dom (Arr) Ob)
(declare-fun Ex (Arr) Bool)
(assert (forall ((f Arr)) (= (comp (id (cod f)) f) f)))
(assert (forall ((f Arr)) (= (comp f (id (dom f))) f)))
(assert (forall ((f Arr) (g Arr) (h Arr)) (= (comp f (comp g h)) (comp (comp f g) h))))
(assert (forall ((A Ob)) (= (dom (id A)) A)))
(assert (forall ((A Ob)) (= (cod (id A)) A)))
(assert (forall ((A Ob)) (Ex (id A))))
(assert (forall ((f Arr) (g Arr)) (= (Ex (comp f g)) (= (dom f) (cod g)))))
(model-add monic!89
           ()
           (Array Arr Bool)
           (lambda ((f Arr))
             (forall ((g Arr) (h Arr))
               (or (not (= (comp f g) (comp f h))) (= h g)))))



AssertionError: unknown

# Metatheory
Termination, confluence, progress, type preservation, consistency, 


Abstract Binding Trees. Can we force Z3 to be unware of the binders by making it an uninterpreted sort with views?
Reminds me of the quotienting construction (like equivalent sequences of CReal). We make a new sort of the quotiented thing and an injecton function and a lift of some equvlaence relation on the domain to an actual equivalence over the quotient sort.
In this case alpha equivalence.

I had some ideas int smt.md about model checking these kinds of problems.

Z3 hoas? It didn't work right?

In [ ]:
# Krishnaswami https://semantic-domain.blogspot.com/2015/03/abstract-binding-trees.html?m=1
# https://www.cs.cmu.edu/~rjsimmon/15312-s14/hws/hw1update2-handout.pdf
Sort("Var")
abt = Sort("abt")
view = Datatype("view", "op", "lam", "var") # no app?
Function("into", )
Function("out",)

Function("subst", "Var", )


In [5]:
from z3 import *
Term = DatatypeSort("Term") #, [("nil", Unit), ("cons", (Int, "List"))])
Term.constructor("Var", IntSort())
Term.constructor("App", [Term, Term])
Term.constructor("Const", IntSort())


TypeError: DatatypeSortRef.constructor() takes 2 positional arguments but 3 were given

### Nominal Sets


In [ ]:
Atom = DeclareSort("Atom")
#Perm = ArraySort(Atom, Atom)
swap = Function("swap", Atom, Atom, Perm)
mul(x,x) = id
mul(swap(a,b), swap(c,d)) = swap(b,a) * swap(c,d)
swap(a,b) == swap(b,a)
app(mul(x,y), z) = app(x, app(y, z))
app(swap(a,b), a) = b
app(swap(a,b), c) = c if a != c and b != c

app(swap(a,b),z) = y ==> z = app(swap(a,b), y)
f(p x, y, z) => p f(x,p y,p z) # equivariance.
p f (x, y , z) <=> f (p x, p y, p y)

fvar(t) = 

# Internal Proofs



In [ ]:
Proof = DeclareSort("Proof")
Function("even", IntSort(), Proof, BoolSort()) # last argument style

index = Function("pindex", IntSort(), Proof)
infer = Function(  )

# SHould proofs be trees
Proof = DeclareSort("Proof")
Proof.constructor("axiom")
Proof.constructor("infer", [Proof, Proof])

# the proof that 2 bool expressions always evaluate to true.
Function("and", BoolSort(), BoolSort(), Proof, BoolSort())
Function("pand", Proof, Proof, Proof)

# Justification
Path = DatatypeSort("Path")
Cons()
path(x,y,P)

# dialectica
# lenses as tactics. Hmmm. scope discipline. Up down cntrol flow.
Function("implies", BoolSort(), Proof, )
